In [1]:
import os
import dotenv

from azure.ai.ml import MLClient
from azure.identity import AzureCliCredential

dotenv.load_dotenv()
subscription = os.getenv(f"subscription_id")
resource_group = os.getenv(f"resource_group")
workspace = os.getenv(f"workspace_name")

ml_client = MLClient(
    AzureCliCredential(), 
    subscription, 
    resource_group, 
    workspace,
)

In [2]:
from azure.ai.ml import command, Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes


# Azure Machine Learning workspace details:
subscription = os.getenv(f"subscription_id")
resource_group = os.getenv(f"resource_group")
workspace = os.getenv(f"workspace_name")
datastore_name = 'workspaceblobstore'
path_on_datastore = 'stroppel_videos/_test/'
output_path_on_datastore = 'stroppel_tracking_output/_test_with_videos_2/'

# long-form Datastore uri format:
uri = f'azureml://subscriptions/{subscription}/resourcegroups/{resource_group}/workspaces/{workspace}/datastores/{datastore_name}/paths/'
uri_input = f'{uri}{path_on_datastore}'
uri_output = f'{uri}{output_path_on_datastore}'

inputs = {
    "job_data_path": Input(type="uri_file",
              path=uri_input + 'start_2023-03-28T00-25-10.036+00-00.mp4',
              mode=InputOutputModes.DOWNLOAD,
              )
}
outputs = {
    "job_output_path": Output(type="uri_folder",
                path=uri_output,
                mode=InputOutputModes.RW_MOUNT,
                )
}

In [3]:
arguments = [
    "--tracking_config", "kalman_tracking_settings.yaml",
    "--job_inputs_path", "${{inputs.job_data_path}}",
    "--job_output_path", "${{outputs.job_output_path}}",
    "--log_level", "INFO",
]

job = command(
    code="./components/kalman_tracking/src/",
    command="python kalman_tracking_azure.py " + " ".join(arguments),
    environment="fishy_environment:6",
    compute="Standard-A1-v2",
    experiment_name="batch_tracking",
    inputs=inputs,
    outputs=outputs,
)

In [4]:
import shutil
# copy library files to job source directory temporarily
pth = './components/kalman_tracking/src/algorithm/'
shutil.copytree('../algorithm/', pth)

returned_job = ml_client.create_or_update(job)

shutil.rmtree(pth)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


# As normal pipeline

In [3]:
from azure.ai.ml import load_component

run_tracking = load_component(source="./components/kalman_tracking/tracking.yml")

In [4]:
from azure.ai.ml.dsl import pipeline


@pipeline(
    compute="Standard-A1-v2",
)
def kalman_tracking_pipeline(
    input_data: Input(type=AssetTypes.URI_FOLDER), 
    output_data: Output(type=AssetTypes.URI_FOLDER) = None,
):
    tracking_results = run_tracking(
        data=input_data,
    )
    
    # tracking_results.outputs.detections = Output(
    #     type="uri_folder",
    #     path=output_uri,
    #     mode=InputOutputModes.RW_MOUNT,
    # )

    return {"detections": tracking_results.outputs.detections}

In [5]:
pipeline_input = Input(type="uri_file",
    path=uri_input,
    mode=InputOutputModes.DOWNLOAD,
)

pipeline_job = kalman_tracking_pipeline(
    input_data=pipeline_input,
)

import shutil
# copy library files to job source directory temporarily
pth = './components/kalman_tracking/src/algorithm/'
shutil.copytree('../algorithm/', pth)

pipeline_job_run = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline-kalman-tracking"
)
shutil.rmtree(pth)
pipeline_job_run

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Experiment,Name,Type,Status,Details Page
pipeline-kalman-tracking,shy_vase_f6fs4hnyzt,pipeline,NotStarted,Link to Azure Machine Learning studio
